### Create a config file
First, we read an initial config


In [ ]:
import os 
from mmengine import Config
cfg = Config.fromfile(r'../configs/OrgSegNet/OrgSeg_PlantCell_768x512.py')

In the next step, we need to modify the config for the training. To accelerate the process, we finetune the model from trained weights.
<br>

Since the given config is used to train OrgSegNet on the PlantCell dataset, we need to modify it accordingly for our new dataset.  




In [ ]:
# Since we use only one GPU, BN is used instead of SyncBN
cfg.norm_cfg = dict(type='BN', requires_grad=True)
cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.decode_head.norm_cfg = cfg.norm_cfg
cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg

# Define the batch_size for train_dataloader\test_dataloader\val_dataloader
cfg.train_dataloader.batch_size = 2
cfg.test_dataloader.batch_size = 1
cfg.val_dataloader.batch_size = 1

# We should define the data_root of new dataset
# Please make sure the dataset is properly structured according the the guidelines.
data_root = r'FinetuneDataset001'
img_dir = 'image'
ann_dir = 'label'

cfg.train_dataloader.dataset.data_root = data_root
cfg.train_dataloader.dataset.data_prefix = dict(img_path=img_dir, seg_map_path=ann_dir)
cfg.train_dataloader.dataset.pipeline = cfg.train_pipeline
cfg.train_dataloader.dataset.ann_file = 'splits/train.txt'


cfg.val_dataloader.dataset.data_root = data_root
cfg.val_dataloader.dataset.data_prefix = dict(img_path=img_dir, seg_map_path=ann_dir)
cfg.val_dataloader.dataset.pipeline = cfg.test_pipeline
cfg.val_dataloader.dataset.ann_file = 'splits/val.txt'


cfg.test_dataloader.dataset.data_root = data_root
cfg.test_dataloader.dataset.data_prefix = dict(img_path=img_dir, seg_map_path=ann_dir)
cfg.test_dataloader.dataset.pipeline = cfg.test_pipeline
cfg.test_dataloader.dataset.ann_file = 'splits/test.txt'

# Load the pretrained weights
cfg.load_from = "../checkpoints/OrgSegNet_iter_Version1.pth"

# Set up working dir to save files and logs.
cfg.work_dir = './work_dirs/finetune001'

# Define the max iterations for the finetune pipline 
cfg.train_cfg.max_iters = 10000

# Validation is performed every 200 iterations
cfg.train_cfg.val_interval = 200

cfg.default_hooks.logger.interval = 10

# The model will be saved every 200 iterations
cfg.default_hooks.checkpoint.interval = 200
# save newest 3 checkpoints
cfg.default_hooks.checkpoint.max_keep_ckpts = 3
# save best checkpoint based mIoU
cfg.default_hooks.checkpoint.save_best = save_best=['mIoU']
cfg.default_hooks.checkpoint.rule = 'greater'

# Set seed to facilitate reproducing the result
cfg['randomness'] = dict(seed=0)
# Let's have a look at the final config used for training
# print(f'Config:\n{cfg.pretty_text}')

### Train

In [ ]:
from mmengine.runner import Runner
runner = Runner.from_cfg(cfg)

In [ ]:
# start training
runner.train()

## Test or Inference

When the fine-tuning pipline was done, We can start the inference pipline with the best model

In [ ]:
# First we should read the newest config from the workdir.
from mmengine import Config
cfg = Config.fromfile(r'./demo/work_dirs/finetunetest/OrgSeg_PlantCell_768x512.py')

In [ ]:
import mmcv
import matplotlib.pyplot as plt
from mmseg.apis import init_model, inference_model, show_result_pyplot, show_cell_pyplot
palette = [[255, 255, 255], [174, 221, 153], [14, 205, 173], [238,137,39], [244, 97, 150]]

# Read and init the model from the config and the best mIoU model (checkpoint) in the workdir.
checkpoint_path = './demo/work_dirs/finetunetest/best_mIoU_iter_xxxxxx.pth'
model = init_model(cfg, checkpoint_path, 'cuda:0')

# Read an image for inference.
img = mmcv.imread('./demo_images/demo2.tif')
result = inference_model(model, img)
plt.figure(figsize=(8, 6))
vis_result = show_cell_pyplot(model, img, result, ogrannel=None)
# Show the image
plt.imshow(mmcv.bgr2rgb(vis_result))
